In [0]:
from pyspark.sql.utils import AnalysisException

In [0]:
catalog_name = dbutils.widgets.get("catalog_name")
schema_names = dbutils.widgets.get("schema_names")
location_details = dbutils.widgets.get("location_details")

In [0]:
print(schema_names)
print(location_details)

In [0]:
schema_names_list = schema_names.split(',')
location_details_list = location_details.split(',')

In [0]:
print(schema_names_list)
print(location_details_list)

In [0]:
def assign_locations_to_schemas(catalog_name, schemas, locations):
    """
    Assign locations to schemas in order.
    Args:
        catalog_name (str): Catalog name
        schemas (list): List of 3 schema names
        locations (list): List of 3 locations (paths or external location names)
    """
    
    if len(schemas) != len(locations):
        raise ValueError(f"Schema count ({len(schemas)}) must match location count ({len(locations)})")
    
    results = []
    
    for schema, location in zip(schemas, locations):
        try:
            # Check if schema exists
            does_schema_exists = check_schema_exists(catalog_name, schema)
            
            if does_schema_exists:
                # Alter existing schema
                recreate_schema_with_location(catalog_name, schema, location)
                results.append(f"DROPPED & RECREATED THE SCHEMA WITH LOCATION: {catalog_name}.{schema} -> {location}")
            else:
                # Create new schema
                create_schema_with_location(catalog_name, schema, location)
                results.append(f"CREATED: {catalog_name}.{schema} -> {location}")
                
        except Exception as e:
            results.append(f"ERROR: {catalog_name}.{schema} -> {str(e)}")
    
    return results


In [0]:
def check_schema_exists(catalog, schema):
    """Check if schema exists."""
    try:
        spark.sql(f"DESCRIBE SCHEMA `{catalog}`.`{schema}`").collect()
        return True
    except AnalysisException:
        return False

In [0]:
def create_schema_with_location(catalog, schema, location):
    """Create schema with location."""
    location_clause = f"MANAGED LOCATION `{location}`" if not location.startswith(('s3://', 'gs://', 'abfss://')) else f"MANAGED LOCATION '{location}'"
    
    spark.sql(f"""
        CREATE SCHEMA IF NOT EXISTS `{catalog}`.`{schema}`
        {location_clause}
    """)

In [0]:
def recreate_schema_with_location(catalog_name, schema_name, location):
    """Recreate schema with location."""    
    try:
        # Drop existing schema if exists
        spark.sql(f"DROP SCHEMA IF EXISTS `{catalog_name}`.`{schema_name}` CASCADE")

        print(f"DROPPED: {catalog_name}.{schema_name}")
        
        # Create new schema with location
        spark.sql(f"""
            CREATE SCHEMA `{catalog_name}`.`{schema_name}`
            MANAGED LOCATION '{location}'
        """)
        print(f"RECREATED: {catalog_name}.{schema_name}")
        return f"RECREATED: {catalog_name}.{schema_name}"
        
    except Exception as e:
        return f"ERROR: {catalog_name}.{schema_name} -> {str(e)}"

In [0]:
print(assign_locations_to_schemas(catalog_name, schema_names_list, location_details_list))